# Gradient Checking

Give me a proof that your backpropagation is actually working!

In [113]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io

from tools.activation_function import sigmoid, ReLU, ReLU_derivative

# 1 - How it works?

### (1) derivative and derivative_approx
$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$

- $y = \theta * x$
- $grad_{true} = \theta$
- $grad_{approx} = \theta \approx$ $\frac{\theta(x + \epsilon) - \theta(x - \epsilon)}{2\epsilon}$


### (2) difference
$$ difference = \frac {\mid\mid grad - gradapprox \mid\mid_2}{\mid\mid grad \mid\mid_2 + \mid\mid gradapprox \mid\mid_2} \tag{2}$$

- $np.linalg.norm([3, 4])$ = $||[3, 4]∣∣_2$ = $(3^2 + 4^2)^{\frac{1}{2}} = 5$

### (3) checking
$difference < epsilon : True$

In [114]:
def forward (x, theta):
    # eg: y = 2x
    return np.dot(theta, x)

def backward (x, theta):
    # eg: y = 2x 导数为2
    return theta

def example_checking (x, theta, epsilon):
    y_plus = forward(x + epsilon, theta)
    y_minus = forward(x - epsilon, theta)
    return (y_plus - y_minus) / (2 * epsilon)

In [115]:
x, theta = 2, 4
epsilon = 1e-7

grad_true = backward(x, theta)
grad_approx = example_checking(x, theta, epsilon)
print('===== 1. 导数 =====', grad_true)
print('===== 1. 导数预估值 =====', grad_approx)
print('===== 2. Difference 计算 =====') 
diff_up = np.linalg.norm(grad_true - grad_approx)
diff_down = np.linalg.norm(grad_true) + np.linalg.norm(grad_approx)
difference = diff_up / diff_down
print('===== 3. 校验值对比 =====') 
if (difference < epsilon):
    print('difference', difference)
    print('The gradient is correct!')
else:
    print('The gradient is wrong!')

===== 1. 导数 ===== 4
===== 1. 导数预估值 ===== 3.9999999978945766
===== 2. Difference 计算 =====
===== 3. 校验值对比 =====
difference 2.6317792396744613e-10
The gradient is correct!


# 2. Deep NN

参考: W1__L2_Regularization.ipynb 

In [116]:
def load_data():
    data = scipy.io.loadmat('data/data.mat')
    train_X = data['X'].T # (2, 211) 
    train_Y = data['y'].T # (1, 211)
    return train_X, train_Y

train_X, train_Y = load_data()

In [177]:
def forward_propagation (X, params):
    # 3 Layers (X -> A1(relu) -> A2(relu) -> A3(sigmoid))
    W1 = params['W1']
    W2 = params['W2']
    W3 = params['W3']
    
    b1 = params['b1']
    b2 = params['b2']
    b3 = params['b3']
    
    # Layer 1
    A1 = ReLU(np.dot(W1.T, X) + b1)
    # Layer 2
    A2 = ReLU(np.dot(W2.T, A1) + b2)
    # Layer 3
    A3 = sigmoid(np.dot(W3.T, A2) + b3)

    A = {
        'A0': X,
        'A1': A1,
        'A2': A2,
        'A3': A3
    }

    return A

def cost_function (A, y):
    A3 = A['A3']
    m = y.shape[1]
    loss = -y * np.log(A3) - (1 - y) * np.log(1 - A3)
    J = (1 / m) * np.sum(loss)

    return J

def backward_propagation (A, y, params):
    m = y.shape[1]
    # dJ_dZ3 = A3 - y
    dJ_dZ3 = A['A3'] - y
    dJ_dW3 = (1 / m) * np.dot(A['A2'], dJ_dZ3.T)
    dJ_db3 = (1 / m) * np.sum(dJ_dZ3, axis = 1, keepdims = True)
    
    # dJ_dZ2
    dJ_dZ2 = np.dot(params['W3'], dJ_dZ3) * ReLU_derivative(A['A2'])
    dJ_dW2 = (1 / m) * np.dot(A['A1'], dJ_dZ2.T)
    dJ_db2 = (1 / m) * np.sum(dJ_dZ2, axis = 1, keepdims = True)
    
    # dJ_dZ1
    dJ_dZ1 = np.dot(params['W2'], dJ_dZ2) * ReLU_derivative(A['A1'])
    dJ_dW1 = (1 / m) * np.dot(A['A0'], dJ_dZ1.T)
    dJ_db1 = (1 / m) * np.sum(dJ_dZ1, axis = 1, keepdims = True)
    
    grads = {
        'dJ_dW3': dJ_dW3,
        'dJ_db3': dJ_db3,
        'dJ_dW2': dJ_dW2,
        'dJ_db2': dJ_db2,
        'dJ_dW1': dJ_dW1,
        'dJ_db1': dJ_db1
    }
    
    return grads
    
def update_derivatives (params, grads, alpha, L_len):
    for l in range(1, L_len):
        params['W' + str(l)] -= alpha * grads['dJ_dW' + str(l)]
        params['b' + str(l)] -= alpha * grads['dJ_db'+ str(l)]
    
    return params

# 3. Gradient Checking

In [178]:
# 将 Weights, bias 平铺
def dictionary_to_vector(parameters):
    keys = []
    count = 0
    for key in ["W1", "b1", "W2", "b2", "W3", "b3"]:
        # flatten parameter
        """
        W1: (2, 20)
        b1: (20, 1)
        
        parameters[key] = [[1, 2], [3, 4]]
        new_vector = [[1], [2], [3], [4]]
        keys = ['W1', 'W1', 'W1', 'W1']
        """
        new_vector = np.reshape(parameters[key], (-1, 1))
        keys = keys + [key] * new_vector.shape[0]
        
        if count == 0:
            theta = new_vector
        else:
            """
              theta = [[1], [2], [3], [4]]
              new_vector = [[5], [6]]
              
              theta = np.concatenate((theta, new_vector), axis=0)
              theta = [[1], [2], [3], [4], [5], [6]]
            """
            theta = np.concatenate((theta, new_vector), axis=0)
        count = count + 1
    return theta, keys

# 将 导数 平铺
def gradients_to_vector (gradients):
    count = 0
    for key in ["dJ_dW1", "dJ_db1", "dJ_dW2", "dJ_db2", "dJ_dW3", "dJ_db3"]:
        new_vector = np.reshape(gradients[key], (-1,1))
        if count == 0:
            theta = new_vector
        else:
            theta = np.concatenate((theta, new_vector), axis=0)
        count = count + 1

    return theta

def vector_to_dictionary(theta):
    parameters = {}
    parameters["W1"] = theta[:6].reshape((2,3))
    parameters["b1"] = theta[6:9].reshape((3,1))
    parameters["W2"] = theta[9:15].reshape((3,2))
    parameters["b2"] = theta[15:17].reshape((2,1))
    parameters["W3"] = theta[17:19].reshape((2,1))
    parameters["b3"] = theta[19:20].reshape((1,1))

    return parameters

def gradient_descent_checking (params, grads, epsilon, X, y):
    """
    1. 平铺所有weights and bias
    所有参数都放在了params, 格式是object / dictionary
    params = {
      'W1': W1,
      'W2': W2,
      'b1': b1,
      'b2': b2
    }

    return [W1, b2, W2, b2, W3, b3, .....]

    2. 平铺 所有dJ_dw, dJ_db
    3. 初始化 J_plus, J_minus, grad_approx
    4. 平铺的w,b 每个都需要计算

    """
    # [1] 平铺所有weights and bias
    theta_arr, keys = dictionary_to_vector(params)
    # [2] 平铺所有dJ_db, dJ_dw
    grad_arr = gradients_to_vector(grads)
    # [3] 初始化 J_plus, J_minus, grad_approx
    param_num = theta_arr.shape[0]
    J_plus, J_minus, grad_approx = np.zeros((param_num, 1)), np.zeros((param_num, 1)),np.zeros((param_num, 1))
    
    # [4] 计算
    for i in range(param_num):
        # (y(x + e) - y(x - e)) / 2e
        # theta_plus
        plus = np.copy(theta_arr)
        plus[i][0] += epsilon
        plus_matrix = vector_to_dictionary(plus)
        
        # y(x + e)
        A_plus = forward_propagation(X, plus_matrix)
        J_plus[i] = cost_function (A_plus, y)
        
        # theta_minus
        minus = np.copy(theta_arr)
        minus[i][0] -= epsilon
        minus_matrix = vector_to_dictionary(minus)
        # y(x - e)
        A_minus = forward_propagation(X, minus_matrix)
        J_minus[i] = cost_function (A_minus, y)
        
        # (y(x + e) - y(x - e)) / 2e
        grad_approx[i] = (J_plus[i] - J_minus[i]) / (2 * epsilon)   
    
    print('===== grad_arr =====')
    print(grad_arr)
    print('===== grad_approx =====')
    print(grad_approx)
    
    numerator = np.linalg.norm(grad_arr - grad_approx)
    denominator = np.linalg.norm(grad_arr) + np.linalg.norm(grad_approx)
    difference = numerator / denominator
    
    if difference > epsilon:
        print("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
    else:
        print("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")
    
    return difference
    

In [179]:
class NN_Model ():
    def __init__(self, X, y, Layers):
        self.X = X
        self.y = y
        
        self.Layers = Layers
        self.L_len = len(Layers)
        self.m = X.shape[1]
        
        # hyperparameters:
        self.alpha = 0.01
        self.iterations = 100000
    
    def init_parameters_he (self):
        parameters = {}
        for i in range(1, len(self.Layers)):
            # 缩放因子
            he = np.sqrt(2. / self.Layers[i - 1])
            parameters['W' + str(i)] = np.random.randn(self.Layers[i - 1], self.Layers[i]) * he
            parameters['b' + str(i)] = np.zeros(shape = (self.Layers[i], 1))
        return parameters

    def training (self):
        X, y = self.X, self.y
        L_len, alpha = self.L_len, self.alpha
        
        J_arr = []
        
        # 1. init parameters
        params = self.init_parameters_he()


        for i in range(self.iterations):
            # 2. Forward propagation
            A = forward_propagation(X, params)

            # 3. Loss
            if i % 100 == 0:
                J = cost_function(A, y)
                J_arr.append(J)

            # 4. backward_propagation
            grads = backward_propagation (A, y, params)
            
            if (i == self.iterations - 1):
                print('====== 梯度检查 start =====')
                gradient_descent_checking (params, grads, 1e-7, self.X, self.y)
                print('====== 梯度检查 end =====')
            
            # 5. update_derivatives
            params = update_derivatives (params, grads, alpha, L_len)
        
        # return Weights and bias
        return params

In [180]:
Layers = [train_X.shape[0], 3, 2, 1]
print(Layers)

NN_he = NN_Model(train_X, train_Y, Layers)
params_he = NN_he.training()

[2, 3, 2, 1]
====== 梯度检查 start =====
===== grad_arr =====
[[ 2.13528362e-04]
 [ 7.16486517e-03]
 [-4.00088837e-05]
 [-3.78436897e-04]
 [ 3.22127618e-03]
 [-6.99643331e-04]
 [ 8.53933126e-04]
 [-1.44541183e-02]
 [ 3.35052949e-03]
 [ 4.98240132e-04]
 [ 1.34667560e-03]
 [ 1.46905427e-03]
 [ 4.13274149e-04]
 [-1.35246270e-04]
 [-3.00073968e-04]
 [ 4.51679861e-04]
 [ 1.90863111e-03]
 [ 1.04268263e-03]
 [ 7.15907399e-04]
 [-1.00224304e-03]]
===== grad_approx =====
[[  14.98565164]
 [  -9.08872396]
 [   0.28550543]
 [   6.34792615]
 [ -10.74532429]
 [  -5.63222072]
 [-107.6000092 ]
 [  25.55294413]
 [ -39.36782381]
 [ -48.7045468 ]
 [ -46.02313121]
 [  -1.57382981]
 [  -1.3488579 ]
 [  -8.04818276]
 [  -7.35960219]
 [ -41.77030529]
 [ -37.24774947]
 [  19.67993073]
 [   6.10979928]
 [  15.61177427]]
There is a mistake in the backward propagation! difference = 0.9999238087187705
====== 梯度检查 end =====
